In [1]:
import pandas as pd
import numpy as np

In [7]:
train_df = pd.read_csv('../../data/data_base/ResultTrain_NonFixedData.csv')
test_df = pd.read_csv('../../data/data_base/ResultTest_NonFixedData.csv')

In [10]:
train_df.head()

,Photo Name,Concentration,Area,Min distance,Mean distance,Max distance,Standart deviation
0,10118,50.0,25426.480467,0,19.055556,36,11.536319
1,20126,50.0,9330.556986,0,9.072464,15,4.153988
2,10428,50.0,345929.601304,0,3.466572,8,2.049517
3,41120,50.0,284.393980,0,1.708333,3,1.135751
4,40844,50.0,884.652546,0,2.547619,4,1.004807


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18900 entries, 0 to 18899
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Photo Name          18900 non-null  int64  
 1   Concentration       18900 non-null  float64
 2   Area                18900 non-null  float64
 3   Min distance        18900 non-null  int64  
 4   Mean distance       18900 non-null  float64
 5   Max distance        18900 non-null  int64  
 6   Standart deviation  18900 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 1.0 MB


In [12]:
train_df.nunique()

Photo Name             6983
Concentration             7
Area                  17067
Min distance              1
Mean distance         15952
Max distance            196
Standart deviation    16726
dtype: int64

In [16]:
X_train = train_df.drop(['Concentration','Photo Name'], axis=1)
X_train

,Area,Min distance,Mean distance,Max distance,Standart deviation
0,25426.480467,0,19.055556,36,11.536319
1,9330.556986,0,9.072464,15,4.153988
2,345929.601304,0,3.466572,8,2.049517
3,284.393980,0,1.708333,3,1.135751
4,884.652546,0,2.547619,4,1.004807
...,...,...,...,...,...
18895,115.195707,0,1.466667,3,0.884433
18896,176929.386300,0,7.490385,15,3.832541
18897,132.334915,0,0.500000,1,0.500000
18898,737.728035,0,4.680851,7,1.739484


In [17]:
y_train = train_df[train_df.columns[1]].astype('int')
y_train

0        50
1        50
2        50
3        50
4        50
         ..
18895    96
18896    96
18897    96
18898    96
18899    96
Name: Concentration, Length: 18900, dtype: int64

In [18]:
X_test = test_df.drop(['Concentration', 'Photo Name'], axis=1)
X_test

,Area,Min distance,Mean distance,Max distance,Standart deviation
0,7488.147277,0,0.704762,2,0.550736
1,70.783658,0,1.000000,2,0.738549
2,21699.267141,0,14.213333,36,9.459565
3,128590.373657,0,4.972414,10,2.854184
4,33130.007465,0,20.486989,41,10.574783
...,...,...,...,...,...
3495,315517.544907,0,20.838292,36,7.671690
3496,22185.639418,0,9.062802,16,3.469803
3497,56644.307597,0,42.128261,83,22.067950
3498,170506.283329,0,48.460403,74,18.626930


In [19]:
y_test = test_df[test_df.columns[1]].astype('int')
y_test

0       50
1       50
2       50
3       50
4       50
        ..
3495    96
3496    96
3497    96
3498    96
3499    96
Name: Concentration, Length: 3500, dtype: int64

In [14]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

In [13]:
cbc = CatBoostClassifier()

In [29]:
catboost_params = {
    "max_depth": [1,2,3,5,7,9],
    "learning_rate": [0.001, 0.01, 0.05, 0.1, 0.3],
    "iterations": [10, 50, 100, 200, 300, 400, 500, 700, 1000, 1500],
    "border_count":[5, 10, 20, 50, 100, 200, 250],
    "l2_leaf_reg": [1,3,5,10,50,100,150],
}

In [31]:
grid_search = GridSearchCV(estimator=cbc,
                           param_grid=catboost_params, 
                           n_jobs= -1,
                           cv=3,
                           scoring='accuracy',
                           error_score=0)
grid_result = grid_search.fit(X_train, y_train)

final_model = cbc.set_params(
    **grid_result.best_params_
)

cbc.fit(X_train, y_train)

y_pred = cbc.predict(X_test)

print(cbc.score(X_test, y_test))

Learning rate set to 0.091923
0:	learn: 1.8072115	total: 59.9ms	remaining: 59.9s
1:	learn: 1.7110448	total: 66.5ms	remaining: 33.2s
2:	learn: 1.6389079	total: 73.3ms	remaining: 24.3s
3:	learn: 1.5822093	total: 79ms	remaining: 19.7s
4:	learn: 1.5359346	total: 85.1ms	remaining: 16.9s
5:	learn: 1.4977258	total: 91.4ms	remaining: 15.1s
6:	learn: 1.4656538	total: 97.4ms	remaining: 13.8s
7:	learn: 1.4405508	total: 104ms	remaining: 12.9s
8:	learn: 1.4092113	total: 110ms	remaining: 12.1s
9:	learn: 1.3835608	total: 117ms	remaining: 11.6s
10:	learn: 1.3614455	total: 122ms	remaining: 11s
11:	learn: 1.3405863	total: 128ms	remaining: 10.5s
12:	learn: 1.3256503	total: 135ms	remaining: 10.2s
13:	learn: 1.3056133	total: 141ms	remaining: 9.91s
14:	learn: 1.2881484	total: 147ms	remaining: 9.69s
15:	learn: 1.2724671	total: 155ms	remaining: 9.51s
16:	learn: 1.2622219	total: 160ms	remaining: 9.24s
17:	learn: 1.2493070	total: 167ms	remaining: 9.1s
18:	learn: 1.2405996	total: 172ms	remaining: 8.88s
19:	learn